##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/audio/transfer_learning_audio"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a> </td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/audio/transfer_learning_audio.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/audio/transfer_learning_audio.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver en GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tutorials/audio/transfer_learning_audio.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar cuaderno</a> </td>
  <td>     <a href="https://tfhub.dev/google/yamnet/1"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">Ver modelo en TF Hub</a> </td>
</table>

# Aprendizaje de transferencia con YAMNet para la clasificación de sonidos ambientales

[YAMNet](https://tfhub.dev/google/yamnet/1) es una red neuronal profunda preentrenada que puede predecir eventos de audio de [521 clases](https://github.com/tensorflow/models/blob/master/research/audioset/yamnet/yamnet_class_map.csv), como risas, ladridos o una sirena.

En este tutorial aprenderá a:

- Cargar y usar el modelo YAMNet para la inferencia.
- Construir un nuevo modelo usando las incrustaciones de YAMNet para clasificar los sonidos de gatos y perros.
- Evaluar y exportar su modelo.


## Importar TensorFlow y otras librerías


Empiece instalando [TensorFlow I/O](https://www.tensorflow.org/io), que le facilitará la carga de archivos de audio del disco.

In [ ]:
!pip install -q "tensorflow==2.11.*"
# tensorflow_io 0.28 is compatible with TensorFlow 2.11
!pip install -q "tensorflow_io==0.28.*"

In [ ]:
import os

from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_io as tfio

## Acerca de YAMNet

[YAMNet](https://github.com/tensorflow/models/tree/master/research/audioset/yamnet) es una red neuronal preentrenada que emplea la arquitectura de convolución separable en profundidad [MobileNetV1](https://arxiv.org/abs/1704.04861). Puede usar una forma de onda de audio como entrada y hacer predicciones independientes para cada uno de los 521 eventos de audio del corpus de [AudioSet](http://g.co/audioset).

Internamente, el modelo extrae "cuadros" de la señal de audio y procesa lotes de estos cuadros. Esta versión del modelo usa cuadros de 0.96 segundos de duración y extrae un cuadro cada 0.48 segundos.

El modelo acepta un Tensor 1-D float32 o un arreglo NumPy que contenga una forma de onda de longitud arbitraria, representada como muestreo monocanal (mono) de 16 kHz en el rango `[-1.0, +1.0]`. Este tutorial incluye código para ayudarle a convertir archivos WAV al formato compatible.

El modelo devuelve 3 salidas, incluidas las puntuaciones de las clases, las incrustaciones (que usará para el aprendizaje por transferencia) y el [espectrograma del logaritmo mel](https://www.tensorflow.org/tutorials/audio/simple_audio#spectrogram). Puede encontrar más detalles [aquí](https://tfhub.dev/google/yamnet/1).

Un uso particular de YAMNet es como extractor de características de alto nivel: la salida incorporada de 1,024 dimensiones. Tomará las características de entrada del modelo base (YAMNet) y las alimentará a su modelo menos profundo, formado por una capa oculta `tf.keras.layers.Dense`. Luego entrenará la red con una pequeña cantidad de datos para la clasificación de audio *sin necesidad de muchos datos etiquetados ni de un entrenamiento de principio a fin. (Esto es similar al [aprendizaje por transferencia para la clasificación de imágenes con TensorFlow Hub](https://www.tensorflow.org/tutorials/images/transfer_learning_with_hub) para más información).*

Primero, probará el modelo y verá los resultados de la clasificación del audio. Luego construirá la canalización de preprocesamiento de datos.

### Cargar YAMNet desde TensorFlow Hub

Va a usar una YAMNet preentrenada de [Tensorflow Hub](https://tfhub.dev/) para extraer las incrustaciones de los archivos de sonido.

Cargar un modelo desde TensorFlow Hub es sencillo: seleccione el modelo, copie su URL y use la función `load`.

Nota: para leer la documentación del modelo, use la URL del modelo en su navegador.

In [ ]:
yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
yamnet_model = hub.load(yamnet_model_handle)

Con el modelo cargado, puede seguir el [tutorial de uso básico de YAMNet](https://www.tensorflow.org/hub/tutorials/yamnet) y descargar un archivo WAV de muestra para ejecutar la inferencia.


In [ ]:
testing_wav_file_name = tf.keras.utils.get_file('miaow_16k.wav',
                                                'https://storage.googleapis.com/audioset/miaow_16k.wav',
                                                cache_dir='./',
                                                cache_subdir='test_data')

print(testing_wav_file_name)

Va a necesitar una función para cargar archivos de audio, que se usará también más adelante al trabajar con los datos de entrenamiento. (Más información sobre la lectura de archivos de audio y sus etiquetas en [Reconocimiento simple de audio](https://www.tensorflow.org/tutorials/audio/simple_audio#reading_audio_files_and_their_labels)).

Nota: El `wav_data` devuelto por `load_wav_16k_mono` ya está normalizado a valores en el rango `[-1.0, 1.0]` (consulte [la documentación de YAMNet sobre el TF Hub](https://tfhub.dev/google/yamnet/1) para saber más).

In [ ]:
# Utility functions for loading audio files and making sure the sample rate is correct.

@tf.function
def load_wav_16k_mono(filename):
    """ Load a WAV file, convert it to a float tensor, resample to 16 kHz single-channel audio. """
    file_contents = tf.io.read_file(filename)
    wav, sample_rate = tf.audio.decode_wav(
          file_contents,
          desired_channels=1)
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

In [ ]:
testing_wav_data = load_wav_16k_mono(testing_wav_file_name)

_ = plt.plot(testing_wav_data)

# Play the audio file.
display.Audio(testing_wav_data, rate=16000)

### Cargue el mapeo de clases

Es importante cargar los nombres de clase que YAMNet es capaz de reconocer. El archivo de mapeo se encuentra en `yamnet_model.class_map_path()` en formato CSV.

In [ ]:
class_map_path = yamnet_model.class_map_path().numpy().decode('utf-8')
class_names =list(pd.read_csv(class_map_path)['display_name'])

for name in class_names[:20]:
  print(name)
print('...')

### Ejecutar inferencia

YAMNet da puntuaciones por clase a nivel de cuadro (es decir, 521 puntuaciones por cada cuadro). Para determinar las predicciones a nivel de clip, las puntuaciones pueden agregarse por clase a través de los cuadros (por ejemplo, usando la agregación media o máxima). Esto se hace a continuación con `scores_np.mean(axis=0)`. Por último, para encontrar la clase mejor puntuada a nivel de clip, se toma el máximo de las 521 puntuaciones agregadas.


In [ ]:
scores, embeddings, spectrogram = yamnet_model(testing_wav_data)
class_scores = tf.reduce_mean(scores, axis=0)
top_class = tf.math.argmax(class_scores)
inferred_class = class_names[top_class]

print(f'The main sound is: {inferred_class}')
print(f'The embeddings shape: {embeddings.shape}')

Nota: El modelo ha deducido correctamente el sonido de un animal. Su meta en este tutorial es aumentar la precisión del modelo para clases específicas. Además, observe que el modelo generó 13 incorporaciones, 1 por cuadro.

## Conjunto de datos ESC-50

El conjunto de datos [ESC-50](https://github.com/karolpiczak/ESC-50#repository-content) ([Piczak, 2015](https://www.karolpiczak.com/papers/Piczak2015-ESC-Dataset.pdf)) es una recolección etiquetada de 2,000 grabaciones de audio ambiental de cinco segundos de duración. El conjunto de datos consta de 50 clases, con 40 ejemplos por clase.

Descárguelo y extráigalo.


In [ ]:
_ = tf.keras.utils.get_file('esc-50.zip',
                        'https://github.com/karoldvl/ESC-50/archive/master.zip',
                        cache_dir='./',
                        cache_subdir='datasets',
                        extract=True)

### Explorar los datos

Los metadatos de cada archivo se especifican en el archivo csv en `./datasets/ESC-50-master/meta/esc50.csv`

y todos los archivos de audio están en `./datasets/ESC-50-master/audio/`

Creará un `DataFrame` de pandas con el mapeo y lo usará para tener una visión más clara de los datos.


In [ ]:
esc50_csv = './datasets/ESC-50-master/meta/esc50.csv'
base_data_path = './datasets/ESC-50-master/audio/'

pd_data = pd.read_csv(esc50_csv)
pd_data.head()

### Filtrar los datos

Ahora que los datos están almacenados en el `DataFrame`, aplique algunas transformaciones:

- Filtre las filas y use sólo las clases seleccionadas: `dog` y `cat`. Si quiere usar otras clases, aquí puede elegir.
- Modifique el nombre del archivo para que contenga la ruta completa. Esto facilitará la carga más adelante.
- Cambie los objetivos para que estén dentro de un rango específico. En este ejemplo, `dog` permanecerá en `0`, pero `cat` pasará a ser `1` en lugar de su valor original de `5`.

In [ ]:
my_classes = ['dog', 'cat']
map_class_to_id = {'dog':0, 'cat':1}

filtered_pd = pd_data[pd_data.category.isin(my_classes)]

class_id = filtered_pd['category'].apply(lambda name: map_class_to_id[name])
filtered_pd = filtered_pd.assign(target=class_id)

full_path = filtered_pd['filename'].apply(lambda row: os.path.join(base_data_path, row))
filtered_pd = filtered_pd.assign(filename=full_path)

filtered_pd.head(10)

### Cargar los archivos de audio y recuperar las incorporaciones

Aquí aplicará el `load_wav_16k_mono` y preparará los datos WAV para el modelo.

Al extraer las incorporaciones de los datos WAV, obtiene un arreglo de forma `(N, 1024)` donde `N` es el número de cuadros que encontró YAMNet (uno por cada 0.48 segundos de audio).

Su modelo usará cada cuadro como una entrada. Por eso, tiene que crear una nueva columna que tenga un cuadro por fila. También tiene que ampliar las etiquetas y la columna `fold` para reflejar adecuadamente estas nuevas filas.

La columna expandida `fold` conserva los valores originales. No se puede mezclar cuadros porque, al dividirse, podría acabar teniendo partes del mismo audio en distintas divisiones, con lo que la validación y la prueba serían menos eficaces.

In [ ]:
filenames = filtered_pd['filename']
targets = filtered_pd['target']
folds = filtered_pd['fold']

main_ds = tf.data.Dataset.from_tensor_slices((filenames, targets, folds))
main_ds.element_spec

In [ ]:
def load_wav_for_map(filename, label, fold):
  return load_wav_16k_mono(filename), label, fold

main_ds = main_ds.map(load_wav_for_map)
main_ds.element_spec

In [ ]:
# applies the embedding extraction model to a wav data
def extract_embedding(wav_data, label, fold):
  ''' run YAMNet to extract embedding from the wav data '''
  scores, embeddings, spectrogram = yamnet_model(wav_data)
  num_embeddings = tf.shape(embeddings)[0]
  return (embeddings,
            tf.repeat(label, num_embeddings),
            tf.repeat(fold, num_embeddings))

# extract embedding
main_ds = main_ds.map(extract_embedding).unbatch()
main_ds.element_spec

### Dividir los datos

Va a usar la columna `fold` para dividir el conjunto de datos en conjuntos de entrenamiento, validación y prueba.

ESC-50 está organizado en cinco `fold` de validación cruzada con tamaño uniforme, para que los clips de la misma fuente original estén siempre en el mismo `fold`. Más información en el artículo [ESC: Conjunto de datos para la clasificación de sonidos ambientales](https://www.karolpiczak.com/papers/Piczak2015-ESC-Dataset.pdf).

El último paso es eliminar la columna `fold` del conjunto de datos, ya que no se usa durante el entrenamiento.


In [ ]:
cached_ds = main_ds.cache()
train_ds = cached_ds.filter(lambda embedding, label, fold: fold < 4)
val_ds = cached_ds.filter(lambda embedding, label, fold: fold == 4)
test_ds = cached_ds.filter(lambda embedding, label, fold: fold == 5)

# remove the folds column now that it's not needed anymore
remove_fold_column = lambda embedding, label, fold: (embedding, label)

train_ds = train_ds.map(remove_fold_column)
val_ds = val_ds.map(remove_fold_column)
test_ds = test_ds.map(remove_fold_column)

train_ds = train_ds.cache().shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.cache().batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.cache().batch(32).prefetch(tf.data.AUTOTUNE)

## Crear su modelo

¡Ya hizo la mayor parte del trabajo! Ahora, defina un modelo [Sequential](https://www.tensorflow.org/guide/keras/sequential_model) muy sencillo con una capa oculta y dos salidas para reconocer gatos y perros a partir de sonidos.


In [ ]:
my_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1024), dtype=tf.float32,
                          name='input_embedding'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(len(my_classes))
], name='my_model')

my_model.summary()

In [ ]:
my_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                 optimizer="adam",
                 metrics=['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(monitor='loss',
                                            patience=3,
                                            restore_best_weights=True)

In [ ]:
history = my_model.fit(train_ds,
                       epochs=20,
                       validation_data=val_ds,
                       callbacks=callback)

Vamos a ejecutar el método `evaluate` en los datos de prueba para asegurarnos de que no hay ningún sobreajuste.

In [ ]:
loss, accuracy = my_model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

¡Excelente!

## Cómo probar su modelo

A continuación, pruebe su modelo sobre la incorporación de la prueba anterior usando sólo YAMNet.


In [ ]:
scores, embeddings, spectrogram = yamnet_model(testing_wav_data)
result = my_model(embeddings).numpy()

inferred_class = my_classes[result.mean(axis=0).argmax()]
print(f'The main sound is: {inferred_class}')

## Guardar un modelo que pueda tomar directamente un archivo WAV como entrada

Su modelo funciona cuando le da las incorporaciones como entrada.

En un escenario real, necesitará usar datos de audio como entrada directa.

Para ello, combinará YAMNet con su modelo en un único modelo que podrá exportar para otras aplicaciones.

Por comodidad a la hora de usar los resultados del modelo, la capa final será una operación `reduce_media`. Si está usando este modelo para el servicio (aprenderá más adelante sobre eso en el tutorial), necesitará el nombre de la capa final. Si no define uno, TensorFlow definirá automáticamente uno incremental, dificultando las pruebas, ya que cambia cada vez que entrena el modelo. Si está usando una operación TensorFlow sin procesar, no puede darle un nombre. Para resolver este problema, creará una capa personalizada que aplique `reduce_media` y la llamará `'classifier'`.


In [ ]:
class ReduceMeanLayer(tf.keras.layers.Layer):
  def __init__(self, axis=0, **kwargs):
    super(ReduceMeanLayer, self).__init__(**kwargs)
    self.axis = axis

  def call(self, input):
    return tf.math.reduce_mean(input, axis=self.axis)

In [ ]:
saved_model_path = './dogs_and_cats_yamnet'

input_segment = tf.keras.layers.Input(shape=(), dtype=tf.float32, name='audio')
embedding_extraction_layer = hub.KerasLayer(yamnet_model_handle,
                                            trainable=False, name='yamnet')
_, embeddings_output, _ = embedding_extraction_layer(input_segment)
serving_outputs = my_model(embeddings_output)
serving_outputs = ReduceMeanLayer(axis=0, name='classifier')(serving_outputs)
serving_model = tf.keras.Model(input_segment, serving_outputs)
serving_model.save(saved_model_path, include_optimizer=False)

In [ ]:
tf.keras.utils.plot_model(serving_model)

Cargue su modelo guardado para verificar que funciona como se espera.

In [ ]:
reloaded_model = tf.saved_model.load(saved_model_path)

Y para la prueba final: dados unos datos de sonido, ¿su modelo devuelve el resultado correcto?

In [ ]:
reloaded_results = reloaded_model(testing_wav_data)
cat_or_dog = my_classes[tf.math.argmax(reloaded_results)]
print(f'The main sound is: {cat_or_dog}')

Si quiere probar su nuevo modelo en un escenario de servicio, puede usar la firma 'serving_default'.

In [ ]:
serving_results = reloaded_model.signatures['serving_default'](testing_wav_data)
cat_or_dog = my_classes[tf.math.argmax(serving_results['classifier'])]
print(f'The main sound is: {cat_or_dog}')


## (Opcional) Unas pruebas más

El modelo está listo.

Comparémoslo con YAMNet en el conjunto de datos de prueba.

In [ ]:
test_pd = filtered_pd.loc[filtered_pd['fold'] == 5]
row = test_pd.sample(1)
filename = row['filename'].item()
print(filename)
waveform = load_wav_16k_mono(filename)
print(f'Waveform values: {waveform}')
_ = plt.plot(waveform)

display.Audio(waveform, rate=16000)

In [ ]:
# Run the model, check the output.
scores, embeddings, spectrogram = yamnet_model(waveform)
class_scores = tf.reduce_mean(scores, axis=0)
top_class = tf.math.argmax(class_scores)
inferred_class = class_names[top_class]
top_score = class_scores[top_class]
print(f'[YAMNet] The main sound is: {inferred_class} ({top_score})')

reloaded_results = reloaded_model(waveform)
your_top_class = tf.math.argmax(reloaded_results)
your_inferred_class = my_classes[your_top_class]
class_probabilities = tf.nn.softmax(reloaded_results, axis=-1)
your_top_score = class_probabilities[your_top_class]
print(f'[Your model] The main sound is: {your_inferred_class} ({your_top_score})')

## Siguientes pasos

Ha creado un modelo que puede clasificar sonidos de perros o gatos. Con la misma idea y un conjunto de datos distinto puede intentar, por ejemplo, construir un [identificador acústico de pájaros](https://www.kaggle.com/c/birdclef-2021/) basado en su canto.

¡Comparta su proyecto con el equipo de TensorFlow en las redes sociales!
